# Predictive Modelling II: Regression

# 1. **Supervised Machine Learning**: Predictions from data with *labels* and *features*  
- Recommendation systems
- Email subject optimization
- Churn prediction

![Overview Machine Learning](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*Z3BdjHruDGtgHBZUgHbpJA.png)
*Source: COGNUB 2016*

***X's and Y's***:
* Unsupervised Learning operates on X's only (i.e., features)
* Supervised Learning requires X's (features) and Y's (labels)


##### **Hint:** Many Names, same thing:

- Features = predictor variables = independent variables = X's
- Target variable = dependent variable = response variable = labels = Y

##### **Important Concept:** We can think of all features (i.e., columns) of an observation (i.e., row) as a ***feature vector*** (of that observation).
- At this point in the course, these features are structured data that are directly interpretable.   
- Later in this course, we will work with ***latent feature vectors*** where features are encoded (or embedded) in vectors. These latent features are not directly intepretable, but are very useful in dimensionality reduction, natural language processing, and AI applications in general.
- You can think of a ***latent feature vector*** as a row (i.e., an observation) with many columns that contain numbers. The meaning of those numbers is, however, not immediately apparent. Yet, there is information embedded in these latent feature vectors that can be of interest/value to us.

In [ ]:
# We first import a number of libraries that we will be using in today's class
import pandas as pd
import numpy as np

# Plotting packages we'll use
import matplotlib.pyplot as plt
import seaborn as sns

# Rather than importing the whole sklearn library, we will import only certain modules
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn import metrics, model_selection
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV

# 2. The Data: Housing Prices in California

In today's class we explore a public dataset on median house prices in blocks in California that can be found already split into training and test sets on your Google Drive in sample_data.  (Information on the original source: https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html)
We'll use a version from the book Hands on Machine Learning by Aurelien Geron, since it adds a category feature, ocean_proximity.

**Today's objective** is to predict housing prices accurately and explore which factors contribute to them most.  

We load the data set below and display a description of the variables.   

**?** Which factors do you think will influence house prices upward and downward?

In [ ]:
# 1. Connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
# 2. Change path to where the class files are: Let's assume you've created a folder 488 on your Google drive where you download Class folders from Canvas
%cd /content/gdrive/MyDrive/488/Class22
# 3. Look what's in the folder
!ls

## 2.1 Read data into a single data frame
Later we will split out the features X from target y = `median_house_value`

In [ ]:
# 1. Read file into a dataframe
df = pd.read_csv("housing.csv")
# 2. Check rows and columns
print(df.shape)
# 3. output first 5 observations
df.head()

## 2.2 Explore Numerical and Categorical Data
With the data collected, it will make sense to do some **feature engineering** -- making new columns by calculations on input columns


### 2.2.1 Numerical data

* `describe()` shows the numeric features, and lists `count`, or number of data entries, the `mean` value, and the `standard deviation`, `min`, `max` and `quartile` values.
* The ranges of feature values differ quite a lot, so we can start to think about whether to normalize, or to eliminate outliers.
* We'll want to make some new per capita or per household features that are more comparable across blocks with different populations.
* We'll use histograms and boxplots to summarize the feature distributions, using the low-level matplotlib to start with. By wrapping the plot code in a function, we can reapply it after changing variables.


In [ ]:
# 1. Get some summary statistics
df.describe()

In [ ]:
# 2. Define a function that creates histograms and boxplots for all numerical variables
def plot_numerics(df): # we loop through all the numeric columns
  for col in df.columns:
    if df[col].dtype == 'float64':
      # and for each column we create space for one row with 2 charts
      f, axes = plt.subplots(1, 2, figsize=(12, 6))
      # our first chart is a histogram and we set the title
      df[col].hist(bins = 50, ax = axes[0])
      axes[0].set_title('Distribution of '+ col)
      # our second column is the boxplot
      df.boxplot(column = col, ax = axes[1])
      # we then use this command to display the charts
      plt.show()

# 3. Call function and pass our dataframe to it
plot_numerics(df)

#### Recap:
* A `histogram` tells is the number of times, or frequency, a value occurs within a `bin`, or bucket, that splits the data (and which we defined). A histogram shows the frequency with which values occur within each of these bins, and can tell us about the distribution of data.
* A `boxplot` captures within the box the `interquartile range`, the range of values from Q1/25th percentile to Q3/75th percentile, and the median value. It also captures the `min` and `max` values of each feature.
* Together, these charts show us the distribution of values for each feature. We can start to make judgements about how to treat the data, for example whether we want to deal with outliers; or whether we want to normalize the data.

### 2.2.2 Dealing with the category data
* `ocean_proximity` contains category data, so we convert it to category type.  
* We can then make a boxplot of the median_house_value by category:
Inland houses clearly appear lower, and island higher, so this may be an important categorical variable.
* On the other hand, we may want to drop the Island category data, since it is only 5 houses.

In [ ]:
# 1. See how many observations we have per category
df.ocean_proximity.value_counts()

In [ ]:
# 2. optional: drop ISLAND
df = df[df['ocean_proximity'] != 'ISLAND']
df.ocean_proximity.value_counts()

In [ ]:
# 3. Typecast Ocean Proximity to categorical
df['ocean_proximity'] = df['ocean_proximity'].astype('category')
df.info()

In [ ]:
# 4. Seaborn can show us the shape of the data with a simple scatter plot
# and also apply categories to hue or style.
plt.figure(figsize=(10,8))
sns.scatterplot(data=df, x='longitude', y='latitude', hue='ocean_proximity', style='ocean_proximity')
plt.show()

In [ ]:
# 5. Let's plot values by category here
f, axes = plt.subplots(1, 1, figsize=(10, 5))
df.boxplot(column='median_house_value', by = 'ocean_proximity', ax = axes)
plt.show()

### 2.2.3 Let's examine our response (or target) variable
* seaborn is a package on top of matplotlib that has a nicer color scheme and works with pandas to make some plots easier. Experiment with plots showing two columns together.
* It seems that blocks whose median house is over $500K were all lumped together.  If we want accurate predictions, we'll have to discard these as outliers. Similarly for houses over 50 years.  

In [ ]:
# 1. Let's visualize the distirubtions of some variables and variable combinations
sns.histplot(data=df, x='median_house_value')
#sns.histplot(data=df, x='median_house_value', y='median_income') # try me
#sns.scatterplot(data=df, x='median_house_value', y='median_income') # try me
#sns.histplot(data=df[df.population>4000], x='median_house_value') #, y='population') # try me
#sns.histplot(data=df, x='median_house_value', y='housing_median_age') # try me
plt.show()

In [ ]:
# 2. We can also look at the counts for each house value (is that really helpful?)
df['median_house_value'].value_counts()

### 2.2.4 Correlation and Multicollinearity

We can generate a heatmap based on the correlation between features (and the response variable) to visually explore which features:  
- might be most relevant in explaining our response variable
- are strongly correlated so that we might not want to include them jointly in our predicition model

**?** What are the features that are most positively correlated?  Why should you not be surprised?

**?** What are the features that are most negatively correlated, and why?  

**?** What correlations with median_house_value are farthest from zero?  Are they what you would expect?

In [ ]:
plt.figure(figsize=(15,12)) # if you need this correlation heatmap to be bigger
corr = df.corr()
sns.heatmap(corr,  vmin = -1, vmax = 1, center = 0, cmap = 'coolwarm', annot = True,
            mask = np.triu(corr));  # masks upper part of triangle

## 2.3 Feature selection, feature transformation, and feature engineering
This is where you have chances to customize what features you want to consider in your model.
* For a small number of records, `total_bedrooms` is not reported.  Since above we saw that 20% of `total_rooms` being bedrooms is typical for the other data, we can impute at this value. (What other ways might you consider imputing?)


* We can replace the category by "One-Hot Encoding". We make dummy or indicator variables for each possibility (1=in that category).  This takes memory, but allows these to be used as numbers in regression.


* The numbers of rooms is closely correlated with population, but rooms per household or rooms per person might be more informative for house price.


In [ ]:
# 1. Missing data: replace nans in total_bedrooms by imputing them to be 20% of all rooms
df.loc[df['total_bedrooms'].isna(), 'total_bedrooms'] = 0.20*df['total_rooms']

In [ ]:
# 2. Add indicator variables for the ocean_proximity values, except the first.
# (When no other is selected, then we know the first must have been selected.)
df = pd.get_dummies(df, columns=['ocean_proximity'], prefix="prox", drop_first=True)
df.info()

In [ ]:
# 3. Save before further feature engineering
df_preFE = df.copy()

In [ ]:
# 4. Engineer some new featurs that might be meaningful
# (Comment out or drop features you don't want to add)

df['Br/Rooms'] = df['total_bedrooms']/df['total_rooms']
#df['Rooms/person'] = df['total_rooms']/df['population']
df['Rooms/hhold'] = df['total_rooms']/df['households']
#df['Br/person'] = df['total_bedrooms']/df['population']
df['Br/hhold'] = df['total_bedrooms']/df['households']
df['pop/hhold'] = df['population']/df['households']

### 2.3.2 Visualize all features again
First, move target column to the end of the data frame to make correlation easier to find.

In [ ]:
# 1. Moves target column
target = 'median_house_value'
cols = list(df.columns.values)
cols.pop(cols.index(target))
cols.append(target) # Ensure target column is at the end
df = df[cols]

In [ ]:
# 2. Call our previously defined function in 2.2.1 and pass it our dataframe to plot histograms and scatterplots of all numerical values
plot_numerics(df)

In [ ]:
# 3. Let's check our variables for correlation
plt.figure(figsize=(15,12)) # if you need this correlation heatmap to be bigger
corr = df.corr()
sns.heatmap(corr,  vmin = -1, vmax = 1, center = 0, cmap = 'coolwarm', annot = True,
            mask = np.triu(corr));  # masks upper part of triangle

In [ ]:
# 4. Drop strongly correlated columns
df.drop(columns = ['total_rooms','total_bedrooms','population','households'], inplace=True)

## 2.4 Preprocess the data

We are in the middle of proprocessing our data to ensure it is a suitable state for modelling.  
Let's review the common preprocessing steps:

* ***Dealing with missing values***  
   where we identify which, if, any missing data we have and how to deal with them.  
   
   For example, we may replace missing values with the mean value for that feature, or by the average of the neighboring values:  
   
    * `pandas` has a number of options for filling in missing data that is worth exploring
    * We can also use `k-nearest neighbor` to help us predict what the missing values should be, or `sklearn Imputer` function (amongst other ways)
    
    
    
* ***Dealing with categorical values***
   by converting them into a numerical representation that can be modelled.  
   
    * There are a number of different ways to do this in `sklearn` and `pandas` such as `pandas.get_dummies`  

* ***Removing outliers***
   to avoid undue influence on the analysis.  Outliers in this data come from many sources:
   * Miscoded data (home values over 500K, age over 50 years)
   * True anomalies from how or what data was gathered (Los Angeles blocks with huge population, vacation areas with huge numbers of rooms per person)
   * Rare circumstances (homes on islands)

* ***Scaling the data***
   to enable different features to be combined or compared, feature data may be transformed to be:
   * all on the same scale (such as within two defined values)
   * (approximately) normally distributed
   * zero-mean
   * etc.  
   
 Not all ML models require this step, but most benefit from it.  Again, `sklearn` and `pandas` have in-built functions to help you do this.

 Finally, we split into Train and Test data, and hide the Test data from any model building efforts.

### 2.4.1 Outlier Removal

Houses over 50 years old or over $500K will not have those parameters accurately recorded, so we should not use them for training or testing.
We can also make a function to identify outliers that are at least three standard deviations from the mean.

In [ ]:
# 1. Let's keep only records with housees under 50 years of age and a value under $500K
print(df.shape)
df = df[(df.housing_median_age <= 50) & (df.median_house_value <=500000)]
print(df.shape)

In [ ]:
# 2. This function can be used on a list of numerical columns to return a list of index values of the outliers
def get_outliers(data, columns, nsd = 3):
    """ Takes a data frame and list of numeric columns names, return the list of indices for which
    a value in any of the columns is more than nsd standard deviations from the column mean."""
    outlier_idxs = []
    for col in set(data.columns).intersection(set(columns)):
        elements = data[col]
        # we get the mean value for each column
        mean = elements.mean()
        # and the standard deviation of the column
        sd = elements.std()
        # we then get the index values of all values higher or lower than the mean +/- nsd standard deviations
        outliers_mask = data[(data[col] > mean + nsd*sd) | (data[col]  < mean  - nsd*sd)].index
        # and add those index values to our list
        outlier_idxs  += [x for x in outliers_mask]
    return list(set(outlier_idxs))

Define the columns where we have identified there could be outliers, in the order you'd like them removed.
**? Any columns you would add?**

In [ ]:
# 3. Defne column list
numeric_columns = ['median_income','median_house_value', 'Br/Rooms', 'Br/person', 'Br/hhold', 'Rooms/person', 'Rooms/hhold', 'pop/hhold']

# 4. Let's check how many observations we have:
print (f"Number of Observations and Features before Cleaning: {df.shape}")

# 5. We call the function we just created on the housing dataset ...
outlier_list = get_outliers(df, numeric_columns)

# 6. ... and drop those records from both our feature and response data
df.drop(outlier_list, axis = 0, inplace=True)

# 7. We can check that this code has worked by looking at the shape of our data
print (f"Number of Observations and Features after Cleaning: {df.shape}")

# WARNING - Be careful: if you repeatedly re-run this cell, you are removing more and more records!

### 2.4.2 Re-Scale the Data

We can create a function to rescale our data so that the mean is zero and standard deviation is unity (1) on all features.  

Lets look at the data before and after re-scaling.  

**!** A good exercise would be to research what StandardScaler does - it is from the scikit-learn library.

In [ ]:
# 1. Check Variable distributions
df.describe()

In [ ]:
# 2. This function loops through columns in a data set and applies the scaler to each that we passed to the function
def scale_numeric(data, numeric_columns, scaler):
    for col in numeric_columns:
        data.loc[:,col] = scaler.fit_transform(data[col].values.reshape(-1, 1))
    return data

In [ ]:
# 3. We can now define the scaler we want to use and apply it to our dataset
scaler = StandardScaler()
numeric_columns = [col for col in df.columns if df[col].dtype == 'float64']
df = scale_numeric(df, numeric_columns, scaler)

# 4. Let's check out varible's distribution after the rescaling
df.describe()

### 2.4.3 Split the Data into X and y, and Train and Test sets
* In order to train our model and see how well it performs, we need to split our data into training and testing sets.  

* We can then train our model on the training set, and test how well it has generalised to the data on the test set.  

* There are a number of options for how we can split the data, and for what proportion of our original data we set aside for the test set.

In [ ]:
# 1. Split out Xs and Ys
housing_y = df['median_house_value'].values
housing_X = df.drop(columns=['median_house_value'])

# 2. A common way for splitting our dataset is using train_test_split
X_train, X_test, y_train, y_test = model_selection.train_test_split(housing_X, housing_y, test_size = 0.2, random_state = 42)

# 3. Get shape of test and training sets
print('Training Set:')
print('Number of records: ', X_train.shape[0])
print('Number of features: ', X_train.shape[1])
print('\n')
print('Test Set:')
print('Number of records: ', X_test.shape[0])
print('Number of features: ', X_test.shape[1])

# 3. Regression Analysis for Prediction
Derived from a notebook accompanying "Classification and Regression in a Weekend" by Ajit Jaokar & Dan Howarth (With contributions from Ayse Mutlu), published by Data Science Central.
Their original notebook is here: https://colab.research.google.com/drive/14m95e5A3AtzM_3e7IZLs2dd0M4Gr1y1W

##  3.1 Linear regression

Linear regression is simple model compared to more complicated regression options, so provides a good baseline.

**Regression Mechanics:**  

y = ax + b
  - y = target (response)  
  - x = single feature  
  - a, b = parameters of model  
  
How do we choose a and b?
  - Define an error function for any given line
  - Choose the line that minimizes the error function
  
**The Loss Function**

Ordinary least squares (OLS): Minimize sum of squares of
residuals

![OLS Loss Function Residuals](https://www.mapxp.app/BUSI488/OLS-residual.jpg "OLS Loss Function")


### 3.1.1 Let's run an OLS on our housing Housing Data!
We start with a single feature, median_income

In [ ]:
# 1. We pull out only one varibale, median_income, from our X_train and X_test sets (which include all variables)
X_income_train = X_train['median_income'].values
X_income_test = X_test['median_income'].values

# 2. Instantiate model: In scikit-learn you create a model separate from your data, which allows you to swap in other models (or data) with minimal code changes
lm = LinearRegression()

# 3. Fit model: we train the model on our training data. Fitting requires both X and y variables for training.
# Because we have only one colum, we reshape it into the format that the linear regressor expects
lm.fit(X_income_train.reshape(-1,1), y_train)

### 3.1.2 Prediction and Performance

Now that we have fit our model, we can use it to predict our response variable in the test set...  
... and check how well our model performed!

In [ ]:
# 1. Predict: We  generate a set of predictions on our unseen features X_test
y_pred = lm.predict(X_income_test.reshape(-1,1))

#### Choose an evaluation metric
* We then need to compare these predictions with the actual result and measure them in some way.  


* This is where the selection of evaluation metric is important. For regression, we measure the distance between the predicted and actual answers in some way. The shorter the distance, the more correct the model is.   


* We cover three common metrics below:  
  
  * `Mean Absolute Error`: which provides a mean score for all the predicted versus actual values as an absolute value   
  
  * `Means Squared Error`: which provides a mean score for all the predicted versus actual values as a square of the absolute value  
  
  * `R2`: which we recommend you research as an exercise to grow your knowledge. WIkipedia and `sklearn` document are a great place to start!  

In [ ]:
# 2. Import a library that we will need for our evaluation metrics
from scipy.stats import norm

# 3. Define a function that evaluates our predictions (y_pred) relative to the ground truth (i.e., y_test)
def evaluate(y_test, y_pred, viz=True):
    # this block of code returns all the metrics we are interested in
    mse = metrics.mean_squared_error(y_test, y_pred)
    msa = metrics.mean_absolute_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)

    print("Mean squared error: ", mse)
    print("Mean absolute error: ", msa)
    print("R^2 : ", r2)
    if viz==True:
      # this creates a chart plotting predicted and actual with distribution of residual
      f, axes = plt.subplots(1, 2, figsize=(14, 8))
      axes[0].scatter(y_test, y_pred)
      axes[0].set(xlabel="median house values: $y_i$", ylabel="Predicted values: $\hat{y}_i$")
      #plt.title("Prices vs Predicted values: $y_i$ vs $\hat{y}_i$")
      axes[1].scatter(x=y_pred, y=y_pred-y_test, )
      axes[1].set(xlabel="residual $y_i-\hat{y}_i$", ylabel="Count", label='resid')

 # 4. Pass the ground truth (y_test) and our predictions (y_pred) to our evaluation function
evaluate(y_test, y_pred, viz=True)

## 3.1.3 Multilinear regression
OLS can use more input variables to get a better prediction

In [ ]:
# 1. Instantiate model
lm = LinearRegression()

# 2. Fit model to full set of variables
lm.fit(X_train, y_train)

# 3. let's save the coefficients and plot them large to small
linear1_coeff = lm.coef_.copy()
coeff_perm = np.flipud(np.argsort(linear1_coeff))

# 4. Plot coefficients
plt.figure(figsize=(12,6))
plt.plot(linear1_coeff[coeff_perm])
plt.ylabel('Coefficients')
plt.xticks(range(housing_X.columns.size), housing_X.columns.values[coeff_perm], rotation=60)
plt.show()

In [ ]:
# 5. Use our trained model to predict the median house values in the test set
y_pred = lm.predict(X_test)

In [ ]:
# 6. Evaluate model performance
evaluate(y_test, y_pred, viz=True)

In [ ]:
#### BONUS ####
# We can explore how metrics are derivied in a little more detail by looking at MAE
# here we will implement MAE using numpy, building it up step by step

# 7a. With MAE, we get the absolute values of the error - as you can see this is of the difference between the actual and predicted values
np.abs(y_test - y_pred)

# 7b. We sum them up
np.sum(np.abs(y_test - y_pred))

# 7c. then divide by the total number of predictions/actual values
# as you will see, we get to the same score implemented above
print (np.sum(np.abs(y_test - y_pred))/len(y_test))

#### 3.1.3.1 YellowBrick
- YellowBrock is a visualization tool for sklearn models, built on top of matplotlib/seaborn.  
- It enables you to easily and quickly explore the performance of different models
- https://www.scikit-yb.org/en/latest/index.html

In [ ]:
# 0. Let's install yellowbrick
# !pip install yellowbrick

# 1. Import some libraries
from yellowbrick.regressor import residuals_plot

In [ ]:
# 2. Use YellowBrick to plot residuals for our original LM model
plt.figure(figsize=(12,6))
# 2b. Fit and plot with yellowbrick
viz = residuals_plot(lm, X_train, y_train, X_test, y_test,  hist=False, qqplot=False)
plt.show()

### 3.1.4 Refine the model
* This step allows us to add or modify features of the datatset. We might do this if, for example, some combination of features better represents the problems space and so is an indicator of the target variable.
* Here, we create one additional feature as an example, but you should reflect on our EDA earlier and see whether there are other features that can be added to our dataset.
* The feature we add is **the log of an existing feature,** so we fit a non-linear model.
* There are other transformations or other algorithms that can fit other non-linear models into this linear framework.  Think about your data to decide what model would work well.  
   
![Types of regression](https://storage.ning.com/topology/rest/1.0/file/get/952937562?profile=original)


In [ ]:
# 0. Go back to dataset before we did out previous preprocessing
log_df = df_preFE.copy() # Restore all fields

# 0a. Missing data: replace nans in total_bedrooms by imputing them to be 20% of all rooms
log_df.loc[log_df['total_bedrooms'].isna(), 'total_bedrooms'] = 0.20*log_df['total_rooms']

# 0b. Remove older houses and those bucketed to $500K
log_df = log_df[(log_df.housing_median_age <= 50) & (log_df.median_house_value <=500000)]

In [ ]:
# 1. Engineer some additional features.
# (Comment out or drop features you don't want to add)

log_df['Br/Rooms'] = log_df['total_bedrooms']/log_df['total_rooms']
#log_df['Rooms/person'] = log_df['total_rooms']/log_df['population']
log_df['Rooms/hhold'] = log_df['total_rooms']/log_df['households']
#log_df['Br/person'] = log_df['total_bedrooms']/log_df['population']
log_df['Br/hhold'] = log_df['total_bedrooms']/log_df['households']
log_df['pop/hhold'] = log_df['population']/log_df['households']

In [ ]:
#### Someone on Kaggle suggested using log to transform skew distributions: https://www.kaggle.com/ilialar/california-housing-analysis-and-preciction

# 2. Define skewed features
skewed_features=['households','median_income','population', 'total_bedrooms', 'total_rooms']

# 3. Keep the names of the log features we will create
log_numerical_features=[]

# 4. Create log features
for f in skewed_features:
    log_df[f + '_log']=np.log1p(log_df[f])
    log_numerical_features.append(f + '_log')

# 5. Drop skewed features
log_df.drop(columns=skewed_features, inplace=True)

In [ ]:
# 6. Remove outliers with previously defined function
numeric_columns = [col for col in log_df.columns if log_df[col].dtype == 'float64']
outlier_list = get_outliers(log_df, numeric_columns)

# 7. and drop those records from both our feature and response data
log_df.drop(outlier_list, axis = 0, inplace=True)

In [ ]:
# 7. Let's scale all numeric columns
scaler = StandardScaler()
numeric_columns = [col for col in log_df.columns if log_df[col].dtype == 'float64']
log_df = scale_numeric(log_df, numeric_columns, scaler)

In [ ]:
# 8. Split out dependent variable (y) and independent variables (X)
log_y = log_df['median_house_value'].values
log_X = log_df.drop(columns=['median_house_value'])

In [ ]:
# 9. Train-Test-Split our Sample
logX_train, logX_test, logy_train, logy_test = model_selection.train_test_split(log_X, log_y, test_size = 0.2, random_state=0)

# 10. Instantiate Model
loglm = LinearRegression()

# 11. Use YellowBrick to fit model and output Performance
plt.figure(figsize=(12,6))
viz = residuals_plot(loglm, logX_train, logy_train, logX_test, logy_test,  hist=False, qqplot=False)
plt.show()

In [ ]:
# 12. let's save the coefficients and plot them large to small
linear2_coeff = loglm.coef_.copy()
coeff_perm = np.flipud(np.argsort(linear2_coeff))

# 13. Plot coefficients
plt.figure(figsize=(12,6))
plt.plot(linear2_coeff[coeff_perm])
plt.ylabel('Coefficients')
plt.xticks(range(log_X.columns.size), log_X.columns.values[coeff_perm], rotation=60)
plt.show()

# 4. Regularization

**Recall:** Linear regression minimizes a loss function
- It chooses a coefficient for each feature variable
- Large coefficients can lead to overfitting
- Penalizing large coefficients is called Regularization


In statistics, there are two critical characteristics of estimators to be considered:
- ***Bias*** is the difference between the true population parameter and the expected estimator
  - It measures the accuracy of the estimates
- ***Variance*** measures the spread, or uncertainty, in these estimates

**Remember:**
- Bias and the variance are desired to be low, as large values result in poor predictions from the model.
- The OLS estimator has the desired property of being unbiased.
- However, it can have a huge variance.

![from KDnuggets](https://www.mapxp.app/BUSI488/bias-variance-tradeoff.jpg "Bias-Variance Trade-Off")


**Solution**: Reduce variance at the cost of introducing some bias.
- This approach is called **regularization** and is almost always beneficial for the predictive performance of the model.

**Principle:**
- Regularization adds a penalty when coefficients get too large in an attempt to reduce the impact of outliers and make models that better predict new values for samples that have not been seen before.
- Important techniques are ridge, lasso, and their combination as elastic net.
- Each has a hyperparameter ($\alpha$) for the size of the penalty
- We will tune these hyperparameters later in this course.

**Rather than go into further details on these techniques, we will apply them to improve model performance!**

If you would like further details you might want to look at DataCamp's excellent tutorial:
[Regularization tutorial](https://www.datacamp.com/community/tutorials/tutorial-ridge-lasso-elastic-net)




## 4.1 Ridge Regression

**Basic Idea:**

- Some predictors may not be as important for out model and we could reduce model complexity by removing them all together.   


- Removing predictors from the model can be seen as settings their coefficients to zero.   


- Instead of forcing them to be exactly zero, let's penalize them if they are too far from zero, thus enforcing them to be small in a continuous way.   


- This way, we decrease model complexity while keeping all variables in the model.  



**Mechanics:**

- Loss function = OLS loss function $+  \alpha \times \sum_{i=1}^n a_i^2$


- Alpha ($\alpha$ ): Parameter we need to choose  


- Picking $\alpha$ here is similar to picking k in k-NN  


- Hyperparameter tuning next section of this notebook  


- $\alpha$ controls model complexity  


- $\alpha$ = 0: We get back OLS (Can lead to overfitting)  


- Very high $\alpha$: Can lead to underfitting

- Assumes the features are standardized and response is centered

**Let's run a ridge regression on our housing Housing Data!**

In [ ]:
# 1. Import Model
from sklearn.linear_model import Ridge

# 2. Instantiate Model
ridge = Ridge(alpha=3)

# 2. Fit Model
ridge.fit(logX_train, logy_train)

# 3. Predict Text
ridge_pred = ridge.predict(logX_test)

# 4. Evaluate Model Performance
ridge.score(logX_test, logy_test)

### 4.1.2 The Effect of Changing Alpha in a Ridge Regression

So how will changes of Alpha impact the effect sizes (of the features) as expresseed by their coefficients?

**Let's try various values for alpha and see!**

In [ ]:
# 1. Define Base model
lm = Ridge(alpha=0)
lm.fit(logX_train, logy_train)

# 2. Create dataframe for our test
ridge_df = pd.DataFrame({'variable': log_X.columns, 'estimate': lm.coef_})
ridge_train_pred = []
ridge_test_pred = []

# 3. Set range of alphas to test
alphas = np.arange(0, 200, 5)

# 4. Create a loop that runs ridge regression with every value of alpha we specified above
for alpha in alphas:
    lm = Ridge(alpha=alpha)
    lm.fit(logX_train, logy_train)
    var_name = 'estimate' + str(alpha)
    ridge_df[var_name] = lm.coef_
    # prediction
    ridge_train_pred.append(lm.predict(logX_train))
    ridge_test_pred.append(lm.predict(logX_test))

# 5. Consolidate results to prepare to plot
ridge_df = ridge_df.set_index('variable').T.rename_axis('estimate').rename_axis(None, axis=1).reset_index()

In [ ]:
# 6. Plot coefficients for all alphas

fig, ax = plt.subplots(figsize=(16, 8))
ridge_df.plot(ax=ax)
ax.axhline(y=0, color='black', linestyle='--')
ax.set_xlabel("Alpha")
ax.set_ylabel("Beta Estimate")
ax.set_title("Ridge Regression Trace", fontsize=16)
ax.legend(labels=ridge_df.columns)
ax.grid(True)

## 4.2 LASSO Regression

- Can be used to select important features of a dataset
- Shrinks the coefficients of less important features to exactly 0

Lasso adds a penalty $\alpha$ times the sum of the absolute value of the $a$ parameters.  

Doing so allows many **parameters that are not needed** in the fitting to be driven to zero.  

Thus, we find out which parameters are necessary for the model.  


**Mechanics:**

- Loss function = OLS loss function $+  \alpha \times \sum_{i=1}^n \left\lvert a_i \right\rvert$


- Alpha ($\alpha$ ): Parameter we need to choose  


- Picking $\alpha$ here is similar to picking k in k-NN  


- Hyperparameter tuning next section of this notebook  


- $\alpha$ controls model complexity  


**Let's run it on our housing Housing Dataset!**


In [ ]:
# 1. Import Model
from sklearn.linear_model import Lasso

# 2. Create placeholder for fittet coefficients
lasso_list = {}

# 3. Loop through values of alpha
for a in [0.01, 0.025, 0.05, 0.075, 0.1, 0.5, 1]:
  # instatiate
  lm = Lasso(alpha = a)
  # fit
  lm.fit(logX_train, logy_train)
  # predict
  y_pred = lm.predict(logX_test)
  print(f'alpha = {a}:')
  # evaluate
  print(lm.score(logX_test, logy_test))
  #evaluate(logy_test, y_pred)
  # Store coefficients
  lasso_list[a] = lm.coef_.copy()

In [ ]:
# 4 Plot effect on coefficient for different alphas in Lasso Regression
plt.figure(figsize=(12,6))
#plt.plot(linear1_coeff[coeff_perm])
#plt.plot(linear2_coeff[coeff_perm])
for item in lasso_list.values():
  plt.plot(item[coeff_perm])
plt.legend(['linear', *lasso_list.keys()])
plt.ylabel('Coefficients')
plt.xticks(range(log_X.columns.size), log_X.columns.values[coeff_perm], rotation=60)
plt.show()

# 5. Different Model: K-Nearest Neighborhood (KNN) Algorithm

> *Show me who your friends are, and I’ll tell you who you are*

The concept of K-NN can hardly be described more simply. This is an old saying, which can be found in many languages and many cultures.


**Basic idea:** Predict the value of a data point by  
- Looking at the values of the ‘k’ closest data points (local values)
- Interpolate these local values to make a prediction  

**Underlying Principle**:
- Find a predefined number (k) training samples closest in distance to a new sample that has to be classified
- The value of the new sample will be defined from these neighbors
- KNN has a fixed user defined constant for the number of neighbors which have to be determined


![A Comparative Study of Linear and KNN Regression](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*1Fqej14RxTPp9zzQ5gKm_g.png "KNN Intuition")
*Image source: https://pub.towardsai.net/a-comparative-study-of-linear-and-knn-regression-a31955e6263d*



In [ ]:
# 1a. Import model
from sklearn.neighbors import KNeighborsRegressor

# 1b. Train-Test-Split our Sample
logX_train, logX_test, logy_train, logy_test = model_selection.train_test_split(log_X, log_y, test_size = 0.2, random_state=42)

# 1c. Instantiate model
kn = KNeighborsRegressor(n_neighbors=7)

# 1d. Use YelloBrick to fit model, evaluate and visualize
plt.figure(figsize=(12,6))
viz = residuals_plot(kn, logX_train, logy_train, logX_test, logy_test,  hist=False, qqplot=False)
plt.show()

### 3.1.6 Location, Location, Location!

![Location!](https://www.socaladvocates.com/wp-content/uploads/2016/03/VenueforBankruptcy-300-300.jpg.webp)

Can location suffice for great predictions?

- Linear Regression

  vs.
- K-NN Regression

> ***Which one do you think will work better?***

In [ ]:
# 1. Location only: Linear Regression

# 1a. Train-Test-Split our Sample and define which variables to use
logX_train, logX_test, logy_train, logy_test = model_selection.train_test_split(log_X, log_y, test_size = 0.2, random_state=21)
latlon = ['latitude','longitude']

# 1b. Instantiate model
linreg = LinearRegression()

# 1c. Use YelloBrick to fit model, evaluate and visualize (we only actually use latlon = ['latitude','longitude'] as the variables)
plt.figure(figsize=(12,6))
viz = residuals_plot(linreg, logX_train[latlon], logy_train, logX_test[latlon], logy_test,  hist=False, qqplot=False)
plt.show()

In [ ]:
# 2. Location only: K-NN

# 2a. Train-Test-Split our Sample and define which variables to use
logX_train, logX_test, logy_train, logy_test = model_selection.train_test_split(log_X, log_y, test_size = 0.2, random_state=21)
latlon = ['latitude','longitude']

# 2b. Instantiate model
kn = KNeighborsRegressor(n_neighbors=7)

# 2c. Use YelloBrick to fit model, evaluate and visualize (we only actually use latlon = ['latitude','longitude'] as the variables)
plt.figure(figsize=(12,6))
viz = residuals_plot(kn, logX_train[latlon], logy_train, logX_test[latlon], logy_test,  hist=False, qqplot=False)
plt.show()

# Recap: Matrices and Solving systems of equations


## 2.2 A Matrix is: A System of Equations

<img src="https://i.imgur.com/K6ySni0.png" alt="prices for apples, bananas, and carrots at 3 different stores" height = 120 align="right">

Matrices provide a compact and convenient way to represent systems of linear equations. Consider the problem of buying apples, bananas, and carrots at different stores with varying prices.


Let 'a' be the quantity of apples, 'b' the quantity of bananas, and 'c' the quantity of carrots. The prices at three stores (Food Lion, Harris Teeter, and a local grocery) can be represented in a matrix. The total cost at each store can be expressed as a linear equation.

**Equations:**

* Food Lion cost = 1.29a + 0.84b + 0.42c
* Harris Teeter cost = 1.24a + 0.86b + 0.41c
* Grocery cost = 1.09a + 0.98b + 0.45c





#### Matrix Representation

We can represent this system of equations in matrix form as: _Prices @ Quantity = Total Cost_.  

$$\left[\matrix{1.29&0.84& 0.42\\
1.24& 0.86& 0.41\\
1.09& 0.98& 0.45}\right]\cdot \left[\matrix{a\\b\\c}\right] = \textit{cost}$$




### Matrix Multiplication

<img src="https://i.imgur.com/t2mHJtN.png" alt="Matrix multiplication: For A*B the number of columns of A must equal the number of rows of B." align="right" height=120>

Matrix multiplication is a fundamental operation. If $A$ is a $j \times k$ matrix and $B$ is a $k \times m$ matrix, then their product $C = A @ B$ is a $j \times m$ matrix. The inner dimensions ($k$) must agree for multiplication to be defined.

**Formula** for all $r\in[0,j)$ and $c\in[0,m)$, calculate $C(r, c) = \sum_{0\le  \ell\lt k} A(r, \ell) \cdot B(\ell, c)$.

**Notes:**
- Numpy uses the @ sign for matrix multiplication; mathematics will use $\cdot$ or *.  
- $A@ B$ is not the same as $B@ A$, and may not even be defined.  Matrix multiplication is not commutative.
- **Why define matrix multiplication this way?**  Laziness.  
 We can write our grocery cost calculations as $A\cdot B = C$, where $A$ is the matrix of prices, $B$ is our vector quantities, and $C$ is the cost. If we know $A$ and $B$, we get costs at all three stores.   
- The quantity should be a column vector. In python, if you use a 1-d vector for the quantity with @, it will figure it out, and give a 1-d vector in return. Experiment below with different quantities of apples, bananas, and carrots.





In [ ]:
import numpy as np

prices = np.array([[1.29, 0.84, 0.42],
                   [1.24, 0.86, 0.41],
                   [1.09, 0.98, 0.45]])

In [ ]:
print(prices @ np.array([[1],[2],[3]]))
print(prices @ np.array([1,2,3]))


## 2.2.1 Solving a System of Equations

A system of linear equations can be written in the form $A\cdot x = b$, where:

* A is the matrix of coefficients.
* x is the column vector of unknowns (variables).
* b is the column vector of constants.

If A is a square matrix (same number of rows and columns) and has full rank, there is generally a unique solution for x. In Python, we can solve for x using `np.linalg.solve(A, b)`.

#### How to solve $Ax=b$?
If $A$ is a single number, $A\ne0$ we can divide both sides by $A$, AKA multiply both sides by the reciprocal $A^{-1}Ax = A^{-1}b$, so $x=A^{-1}b$.  
Exactly this works for any *non-singular* matrix $A$.

**Example:**

Continuing the grocery example, suppose we want to find the quantities of apples, bananas, and carrots that would cost $\$10$ at Food Lion and Harris Teeter, and $\$ 11$ at the local grocery.

Experiment with other price totals. (For some totals you may find yourself having to sell produce to the stores.)


In [ ]:
totals = np.array([10, 10, 11])

quantities = np.linalg.solve(prices, totals)
print("Quantities of apples, bananas, and carrots:\n", quantities)
print("\nCheck: prices @ quantities = \n", prices @ quantities) # @ is matrix multiplication



### Static Equilibrium (Balanced Forces)


<img src="https://imgur.com/rcKTVXE.png" alt="a triangle shaped bridge ABC with a load at B" align="right" height = 220>

Here's a more realistic scenario where we want to find quantities that achieve a desired total.  Consider a triangular bridge ABC:

* A is fixed.
* 10N load is applied at B.
* C rolls.

We want to find the forces $f_1$, $f_2$, and $f_3$ on the bars (positive for expansion, negative for contraction). The horizontal and vertical forces must balance at each vertex to prevent movement or collapse. Let's find three equations for these three variables.




#### Static Equilibrium Equations
<img src="https://imgur.com/tWcvMuq.png" alt="forces on a triangle shaped bridge" align="right" height = 210>

* $A$ is fixed (no equations needed).
* $B$ may move:
    - $B_h$ (horizontal): $f_1\cos(\theta) - f_2\cos(\gamma) = 0$
    - $B_v$ (vertical): $f_1\sin(\theta) + f_2\sin(\gamma) - \textit{Load} = 0$
* $C$ may move horizontally:
    * $C_h$: $f_3 + f_2\cos(\gamma) = 0$

These equations can be written in matrix form as M @ f = b, where we know $M$ and $b$, once we choose the angles for the bridge.  I'll start with an equilateral triangle, but experiment by setting $\theta=45$ or even $\theta=30$.




In [ ]:
import numpy as np
import math

def bridge_forces(thetaDeg=60, gammaDeg=60, Load = 10):
  theta = math.radians(thetaDeg)  # angles to radians
  gamma = math.radians(gammaDeg)
  M = np.array([
      [np.cos(theta), -np.cos(gamma), 0],
      [np.sin(theta), np.sin(gamma), 0],
      [0, np.cos(gamma), 1]
  ])

  b = np.array([0, Load, 0])

  return np.linalg.solve(M, b) # forces


In [ ]:
print("Forces:\n", bridge_forces())
# f_1 & f_2 push up to balance Load, and thus push apart, countered by contraction of f_3.


#### Exercise:

plot the values returned from bridge_forces(th) for th in [5 to 110 by 5s].  Use a lineplot with legend f1, f2, f3, putting th on the x axis. Explain what is happening in the plot.


## 2.2.2 Non-square Systems of Equations

* Rectangular matrices represent non-square systems.
* Underdetermined systems have fewer equations than variables.
* Overdetermined systems have more equations than variables.
* `np.linalg.solve` solves Ax = b "in the least-squares sense" for overdetermined systems, finding the x that minimizes the squared residual: $||Ax - b||^2$.



### Example: Fitting a Line
<img src="https://imgur.com/9Ab0CPr.png" alt="observed values from a riction experiment" >

**Data:** Friction in response to an applied force.


We want to fit a line ($y = mx + c$) to this data and evaluate the error.

<img src="https://imgur.com/5BUVNUR.png" alt="matrix form of equaations for friction" align="right" height=180>

That is, we want:  
$\hphantom{0}5.84 \approx m\cdot 1+c$  
$\hphantom{0}8.87 \approx m\cdot 2+c$  
$11.76 \approx m\cdot 3+c$  
&emsp;&emsp;&emsp;&emsp;$\vdots$  
$29.18 \approx m\cdot 10+c$


In [ ]:
import numpy as np

A = np.column_stack([(1,2,3,4,5,6,7,8,9,10), np.ones(10)])
b = np.array([5.84, 8.87, 11.76, 13.71, 14.52, 17.71, 21.80, 24.25, 25.23, 29.18])


In [ ]:
#prompt: write plot_fit(x,y, m, c) scatterplot points for input vectors x and, y, then nd draw the line y=mx+b for the inputvalues m,c.  Draw the residuals and sum their squared lengths.
import numpy as np
import matplotlib.pyplot as plt

def plot_fit(x,y, m, c):
  # Scatterplot the points
  plt.scatter(x, y, label='Observed Data')

  # Draw the fitted line
  x_line = np.array([min(x), max(x)])
  y_line = m * x_line + c
  plt.plot(x_line, y_line, color='red', label='Fitted Line')

  # Calculate and draw the residuals
  residuals = y - (m * A[:, 0] + c)
  plt.vlines(x, y, y-residuals, color='green', linestyle='dashed', label='Residuals')

  # Calculate the sum of squared residuals
  sum_squared_residuals = np.sum(residuals**2)
  plt.text(6, 5, f'Sum of Squared Residuals: {sum_squared_residuals:.2f}', color='purple')

  # Add labels and legend
  plt.xlabel('Applied Force')
  plt.ylabel('Friction')
  plt.title('Least Squares Fit')
  plt.legend()
  plt.grid(True)
  plt.show()

#### Experiment
Change the last two numbers to check different lines $y=mx+b$.
Residuals are the distance between the point on the line $(mx_i+c) - y_i$, usually with the absolute value or squared to make each residual positive.  

In [ ]:
plot_fit(A[:, 0], b, 2, 5)

#### Best fit line in least-squares sense

Suppose we want to minimize the squared residual: $\textit{resid}^2 = {(Ax-b)}^2 = (Ax-b)^T(Ax-b)= x^TA^TAx-x^TA^Tb -b^TAx +b^Tb$.  
Check the shape: this is a number.  We minimize any function $f(x)$ by taking a derivative $\frac d{dx} f(x) =0$.  
$\frac d{dx} \textit{resid}^2 = A^TAx + x^TA^TA-A^Tb -b^TA = 0$ by solving $A^TAx=A^Tb$.  
Note that this is a square matrix with the same number of equations as unknowns, so we converted an overconstrained system to optimize to a solveable system.

In [ ]:
# Solve for x in Ax = b using least squares
x = np.linalg.lstsq(A, b, rcond=None)[0]
m, c = x
plot_fit(A[:, 0], b, m, c)
print(f"Best line: h = {m}x + {c}")

# Solve A^T * A * x = A^T * b for x
x = np.linalg.solve(A.T @ A, A.T @ b)
m2, c2 = x
print(f"linalg.solve: m = {m2}, c = {c2}, error to previous {np.abs(m-m2)+np.abs(c-c2)}")


### Least Squares Reprise

* Least squares requires a mathematical model, parameters to be found, and observed data.
* Linear least squares problems can be written as Ax = b, where A and b are known, and x is the vector of unknowns.
* The error is the squared residual: error = ||Ax - b||^2$



### Fitting Other Models: Paris/Rio Temperatures

Let's model the annual temperature cycle for Paris and Rio.

The annual cycle looks like a sine wave, so we can try to fit a sine curve to the data.

For months $m = 1:12$, consider a model of the form:

$temp = A + B\cos(m\pi/6) + C\sin(m\pi/6)$

We want to find the values of A, B, and C that best fit the temperature data.  This gives us a system of equations.  For example, for ParisHi, we have:

$55 = A + B\cos(1\pi/6) + C\sin(1\pi/6)$  
$55 = A + B\cos(2\pi/6) + C\sin(2\pi/6)$  
$59 = A + B\cos(3\pi/6) + C\sin(3\pi/6)$  
...

And so on.  We can write this as a matrix equation $M @ [[A], [B],[C]] = temp$.


In [ ]:
import numpy as np

parisHi = np.array([55, 55, 59, 64, 68, 75, 81, 81, 77, 70, 63, 55])
parisLo = np.array([39, 41, 45, 46, 55, 61, 64, 64, 61, 54, 49, 41])

rioHi = np.array([84, 85, 83, 80, 77, 76, 75, 76, 75, 77, 79, 82])
rioLo = np.array([73, 73, 72, 69, 66, 64, 63, 64, 65, 66, 68, 71])

temp_rows = np.array([parisHi, parisLo, rioHi, rioLo])
temp_legend = ['ParisHi', 'ParisLow', 'RioHi', 'RioLow']
temp = temp_rows.transpose() # Swap rows and columns


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

months = np.arange(1, 13)
CON = np.ones(12)
COS = np.cos(2*np.pi/12 * months)
SIN = np.sin(2*np.pi/12 * months)

M = np.column_stack([CON, COS, SIN])
fit_params = np.linalg.lstsq(M, temp)[0]

plt.plot(months, temp, '+', label=temp_legend)
plt.plot(months, M @ fit_params, label=["parisHi_fit", "parisLo_fit", "rioHi_fit", "rioLo_fit"])
plt.legend(temp_legend + ["parisHi_fit", "parisLo_fit", "rioHi_fit", "rioLo_fit"])
plt.xlabel('Month')
plt.ylabel('Temperature')
plt.show()




#### Error in Fitting Sine & Cosine

We can calculate the residual and error for this fit, too.  Since we fit four curves at once (one for each of "parisHi", "parisLo", "rioHi", and "rioLo"), we get four error values.


In [ ]:
resid = M @ fit_params - temp
error = np.sum(resid**2, axis=0)

print("Residuals:\n", resid)
print("Errors:\n", error) # Four errors, one for each city


This notebook draws on  Kyoosik Kim, 2 jan 2019, https://towardsdatascience.com/ridge-regression-for-better-usage-2f19b3a202db